In [68]:
!pip install spark 
!pip install pyspark 
!pip install findspark
!pip show pypspark

In [69]:
import findspark

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark DataFrame example").config("spark.some.config.option", "value").getOrCreate()
#spark = SparkSession.builder.master("local[1]").appName("Spark standalone song word count").getOrCreate()

In [70]:
import os 
workspacepath = os.getcwd()
print("workspace path =      ",workspacepath)
file_location_path ='/var/lib/docker/volumes/hadoop-distributed-file-system/_data'
print("file location path = ",file_location_path) # docker volume 
files_distination_path = '/home/project_Houcine_Slimi'
print('files copying to :  ',files_distination_path) 

workspace path =       /opt/workspace
file location path =  /var/lib/docker/volumes/hadoop-distributed-file-system/_data
files copying to :   /home/project_Houcine_Slimi


# Creation du fichier text song.txt
### C'est les paroles de la chanson "Banana pancakes"

In [71]:
song ="""can't you see that it's just rainin'
Ain't no need to go outside
But baby
You hardly even notice
When I try to show you this
Song is meant to keep you
Doin' what you're supposed to
Wakin' up too early
Maybe we could sleep in
Make you banana pancakes
Pretend like its the weekend now
We could pretend it all the time
Can't you see that it's just rainin'
There ain't no need to go outside
Just maybe
Halaka ukulele mama made a baby
Really don't mind the practice
Cause you're my little lady
Lady lady love me
Cause I love to lay here lazy
We could close the curtains
Pretend like there's no world outside
We could pretend it all the time
And can't you see that it's just rainin'
There ain't no need to go outside
Ain't no need ain't no need
Mmm, mmm, mmm
Can't you see can't you see
Rain all day and I don't mind
But the telephone's singing ringing
It's too early don't pick it up
We don't need to
We got everything we need right here
And everything we need is enough
Just so easy when the whole world fits inside of your arms
Do we really need to pay attention to the alarm
Wake up slow
Mmm wake up slow
But baby
You hardly even notice
When I try to show you this
Song is meant to keep you
From doin' what your supposed to
Wakin' up too early
Maybe we could sleep in
Make you banana pancakes
Pretend like it's the weekend now
We could pretend it all the time
Can't you see that it's just rainin'
There ain't no need to go outside
Ain't no need, ain't no need
Rain all day and I really really really don't mind
Can't you see? Can't you see?"""


with open('song.txt', 'w') as f:
    f.write(song)

In [72]:
song_txt_RDD = spark.sparkContext.textFile("song.txt").collect()
song_txt_RDD
#logRDD= spark.read.csv("song.txt")
#logRDD.collect()

["can't you see that it's just rainin'",
 "Ain't no need to go outside",
 'But baby',
 'You hardly even notice',
 'When I try to show you this',
 'Song is meant to keep you',
 "Doin' what you're supposed to",
 "Wakin' up too early",
 'Maybe we could sleep in',
 'Make you banana pancakes',
 'Pretend like its the weekend now',
 'We could pretend it all the time',
 "Can't you see that it's just rainin'",
 "There ain't no need to go outside",
 'Just maybe',
 'Halaka ukulele mama made a baby',
 "Really don't mind the practice",
 "Cause you're my little lady",
 'Lady lady love me',
 'Cause I love to lay here lazy',
 'We could close the curtains',
 "Pretend like there's no world outside",
 'We could pretend it all the time',
 "And can't you see that it's just rainin'",
 "There ain't no need to go outside",
 "Ain't no need ain't no need",
 'Mmm, mmm, mmm',
 "Can't you see can't you see",
 "Rain all day and I don't mind",
 "But the telephone's singing ringing",
 "It's too early don't pick it up

# Processing : éliminer tout les caractéres non alphabetiques à part les " ' " et transformer en miniscules

In [73]:
# RDD 
song_txt_RDDprocessed_tolist   = spark.sparkContext.parallelize(song_txt_RDD).mapPartitions(lambda iterOfElements : [e.replace('?','').
                                                                                                               replace(',', '').
                                                                                                               replace("'t",' not').
                                                                                                               replace("'s",' is').
                                                                                                               replace("'re",' are').lower() for e in iterOfElements]).collect()
song_txt_RDDprocessed_tolist   = spark.sparkContext.parallelize(song_txt_RDDprocessed_tolist).mapPartitions(lambda iterOfElements : [e.replace('don','do').
                                                                                                                                     replace("'",'g').
                                                                                                                                     replace('casue','because')for e in iterOfElements]).collect()
# list to RDD
song_txt_processedRDD          = spark.sparkContext.parallelize(song_txt_RDDprocessed_tolist)
song_txt_RDDprocessed_tolist

['can not you see that it is just raining',
 'ain not no need to go outside',
 'but baby',
 'you hardly even notice',
 'when i try to show you this',
 'song is meant to keep you',
 'doing what you are supposed to',
 'waking up too early',
 'maybe we could sleep in',
 'make you banana pancakes',
 'pretend like its the weekend now',
 'we could pretend it all the time',
 'can not you see that it is just raining',
 'there ain not no need to go outside',
 'just maybe',
 'halaka ukulele mama made a baby',
 'really do not mind the practice',
 'cause you are my little lady',
 'lady lady love me',
 'cause i love to lay here lazy',
 'we could close the curtains',
 'pretend like there is no world outside',
 'we could pretend it all the time',
 'and can not you see that it is just raining',
 'there ain not no need to go outside',
 'ain not no need ain not no need',
 'mmm mmm mmm',
 'can not you see can not you see',
 'rain all day and i do not mind',
 'but the telephone is singing ringing',
 'it i

In [76]:
counts = song_txt_processedRDD.flatMap(lambda line: line.split(" ")) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)
counts.saveAsTextFile("song_result.txt")
counts.collect()

[('can', 8),
 ('not', 21),
 ('you', 18),
 ('see', 8),
 ('that', 4),
 ('it', 10),
 ('is', 11),
 ('just', 6),
 ('raining', 4),
 ('ain', 8),
 ('no', 9),
 ('need', 12),
 ('to', 14),
 ('go', 4),
 ('outside', 5),
 ('but', 3),
 ('baby', 3),
 ('hardly', 2),
 ('even', 2),
 ('notice', 2),
 ('when', 3),
 ('i', 5),
 ('try', 2),
 ('show', 2),
 ('this', 2),
 ('song', 2),
 ('meant', 2),
 ('keep', 2),
 ('doing', 2),
 ('what', 2),
 ('are', 2),
 ('supposed', 2),
 ('waking', 2),
 ('up', 5),
 ('too', 3),
 ('early', 3),
 ('maybe', 3),
 ('we', 11),
 ('could', 6),
 ('sleep', 2),
 ('in', 2),
 ('make', 2),
 ('banana', 2),
 ('pancakes', 2),
 ('pretend', 6),
 ('like', 3),
 ('its', 1),
 ('the', 10),
 ('weekend', 2),
 ('now', 2),
 ('all', 5),
 ('time', 3),
 ('there', 4),
 ('halaka', 1),
 ('ukulele', 1),
 ('mama', 1),
 ('made', 1),
 ('a', 1),
 ('really', 5),
 ('do', 6),
 ('mind', 3),
 ('practice', 1),
 ('cause', 2),
 ('my', 1),
 ('little', 1),
 ('lady', 3),
 ('love', 2),
 ('me', 1),
 ('lay', 1),
 ('here', 2),
 ('la

In [28]:
#comptage le nombre de mots totat
result = counts.groupByKey().collect()
s = 0 
for r in result:
    s = s + list(r[1])[0]
print("1/ La chanson contient",s, "mots.")

#comptage du nombre de mots distinctes
print("2/ Il existe",len(counts.collect()),"mots distinctes.")

#recurrence de chaque mot 
df = counts.toDF(['Mot' ,'Recurrence'])
df.show()

1/ La chanson contient 337 mots.
2/ Il existe 100 mots distinctes.
+-------+----------+
|    Mot|Recurrence|
+-------+----------+
|    can|         8|
|    not|        21|
|    you|        18|
|    see|         8|
|   that|         4|
|     it|        10|
|     is|        11|
|   just|         6|
|raining|         4|
|    ain|         8|
|     no|         9|
|   need|        12|
|     to|        14|
|     go|         4|
|outside|         5|
|    but|         3|
|   baby|         3|
| hardly|         2|
|   even|         2|
| notice|         2|
+-------+----------+
only showing top 20 rows



In [29]:
df.sort(df.Recurrence.desc()).show(101)

+----------+----------+
|       Mot|Recurrence|
+----------+----------+
|       not|        21|
|       you|        18|
|        to|        14|
|      need|        12|
|        is|        11|
|        we|        11|
|       the|        10|
|        it|        10|
|        no|         9|
|       can|         8|
|       see|         8|
|       ain|         8|
|   pretend|         6|
|      just|         6|
|        do|         6|
|     could|         6|
|        up|         5|
|   outside|         5|
|    really|         5|
|         i|         5|
|       all|         5|
|       and|         4|
|       mmm|         4|
|        go|         4|
|      that|         4|
|   raining|         4|
|     there|         4|
|      when|         3|
|      like|         3|
|     early|         3|
|      baby|         3|
|       too|         3|
|       but|         3|
|      time|         3|
|      lady|         3|
|     maybe|         3|
|      mind|         3|
|  pancakes|         2|
|    hardly|    